# Agent 간단 평가

Dify에서 제작한 에이전트를 가져다가 평가합니다.

Dify에서 에이전트를 제작한 뒤, Batch Run을 통하여 빠르게 결과 값을 뽑아볼 수 있습니다.

In [1]:
import pandas as pd
import os
import pathlib

root_dir = pathlib.PurePath(os.path.dirname(os.getcwd())).parent
data_dir = os.path.join(root_dir, 'data')

In [3]:
# 에이전트 평가 결과를 불러옵니다.
result_df = pd.read_csv(os.path.join(data_dir, 'agent', 'dify_result.csv'))

In [4]:
result_df.head()

,title,short_description,Completion result
0,[코드잇 스프린트 Node.js 백엔드 1기] 위클리 페이퍼 - 1주차,Q1. URL의 구성 요소 중 쿼리가 무엇인지 실제 웹 서비스의 URL을 예시로 설...,"{""class_name"":""Backend""}"
1,FSD 간단하게 시작해보기,공식홈피를 기반 했지만 제 입맛에 따라 해석 했음을 알려드립니다.\n\n프롤로그\n...,"{""class_name"":""Backend""}"
2,[면접을 위한 CS 전공지식 노트] 2.3 네트워크 기기,면접을 위한 CS 전공지식 노트 - 2.3 네트워크 기기,"{""class_name"":""CS""}"
3,🎓 Reinforcement learning - 1,"🎓 강화학습(Reinforcement Learning, RL)이란?\n>강화학습(R...","{""class_name"":""AI""}"
4,TIL-2,논리적으로 제품을 만들 수 있도록 도와주는 프레임워크 중 하나.사용자에 대한 이해를...,"{""class_name"":""DevOps""}"


In [9]:
import ast
result_df['sector'] = result_df['Completion result'].apply(lambda x: ast.literal_eval(x)["class_name"])

In [10]:
result_df.head()

,title,short_description,Completion result,sector
0,[코드잇 스프린트 Node.js 백엔드 1기] 위클리 페이퍼 - 1주차,Q1. URL의 구성 요소 중 쿼리가 무엇인지 실제 웹 서비스의 URL을 예시로 설...,"{""class_name"":""Backend""}",Backend
1,FSD 간단하게 시작해보기,공식홈피를 기반 했지만 제 입맛에 따라 해석 했음을 알려드립니다.\n\n프롤로그\n...,"{""class_name"":""Backend""}",Backend
2,[면접을 위한 CS 전공지식 노트] 2.3 네트워크 기기,면접을 위한 CS 전공지식 노트 - 2.3 네트워크 기기,"{""class_name"":""CS""}",CS
3,🎓 Reinforcement learning - 1,"🎓 강화학습(Reinforcement Learning, RL)이란?\n>강화학습(R...","{""class_name"":""AI""}",AI
4,TIL-2,논리적으로 제품을 만들 수 있도록 도와주는 프레임워크 중 하나.사용자에 대한 이해를...,"{""class_name"":""DevOps""}",DevOps


In [5]:
# ground truth와 concat 후 agent 성능을 평가합니다.
gt_df = pd.read_csv(os.path.join(data_dir, 'agent', 'annotate_data.csv'))

In [6]:
gt_df.head()

,annotation_id,annotator,created_at,id,lead_time,sector,short_description,title,updated_at
0,101,1,2025-01-29T05:54:36.686534Z,1,10.400,Backend,Q1. URL의 구성 요소 중 쿼리가 무엇인지 실제 웹 서비스의 URL을 예시로 설...,[코드잇 스프린트 Node.js 백엔드 1기] 위클리 페이퍼 - 1주차,2025-01-29T05:54:36.686553Z
1,102,1,2025-01-29T05:54:43.279755Z,2,6.479,Frontend,공식홈피를 기반 했지만 제 입맛에 따라 해석 했음을 알려드립니다.\n\n프롤로그\n...,FSD 간단하게 시작해보기,2025-01-29T05:54:43.279774Z
2,103,1,2025-01-29T05:54:45.477998Z,3,2.074,CS,면접을 위한 CS 전공지식 노트 - 2.3 네트워크 기기,[면접을 위한 CS 전공지식 노트] 2.3 네트워크 기기,2025-01-29T05:54:45.478017Z
3,104,1,2025-01-29T05:54:47.032793Z,4,1.430,AI,"🎓 강화학습(Reinforcement Learning, RL)이란?\n>강화학습(R...",🎓 Reinforcement learning - 1,2025-01-29T05:54:47.032811Z
4,105,1,2025-01-29T05:54:51.911492Z,5,4.753,etc,논리적으로 제품을 만들 수 있도록 도와주는 프레임워크 중 하나.사용자에 대한 이해를...,TIL-2,2025-01-29T05:54:51.911510Z


In [7]:
gt_df.sector.unique()

array(['Backend', 'Frontend', 'CS', 'AI', 'etc', 'DevOps'], dtype=object)

In [11]:
result_df.sector.unique()

array(['Backend', 'CS', 'AI', 'DevOps', 'etc', 'Frontend'], dtype=object)

In [14]:
set(gt_df.sector.unique()) == set(result_df.sector.unique())

True

In [15]:
concat_df = pd.DataFrame({
	'gt': gt_df.sector,
	'pred': result_df.sector,
})

In [16]:
concat_df.head()

,gt,pred
0,Backend,Backend
1,Frontend,Backend
2,CS,CS
3,AI,AI
4,etc,DevOps


## Accuracy, Precision, Recall, F1 Score 계산

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(concat_df['gt'], concat_df['pred'])
precision = precision_score(concat_df['gt'], concat_df['pred'], average='macro')
recall = recall_score(concat_df['gt'], concat_df['pred'], average='macro')
f1 = f1_score(concat_df['gt'], concat_df['pred'], average='macro')

In [19]:
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Accuracy: 0.68
Precision: 0.6621371460081137
Recall: 0.7111476758535581
F1 Score: 0.606504659539374


### AI Sector에 대한 분류 성능 평가

In [20]:
# AI Sector에 대한 분류 성능을 평가합니다.
ai_df = concat_df[concat_df['gt'] == 'AI']

In [24]:
accuracy = accuracy_score(ai_df['gt'], ai_df['pred'])
precision = precision_score(ai_df['gt'], ai_df['pred'], average='macro')
recall = recall_score(ai_df['gt'], ai_df['pred'], average='macro')
f1 = f1_score(ai_df['gt'], ai_df['pred'], average='macro')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
